In [67]:
print("womp womp")

womp womp
time: 0 ns (started: 2023-12-15 10:53:50 -05:00)


In [68]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2023-12-15 10:53:50 -05:00)


In [69]:
file_path = r"C:\Users\aaron\Documents\Python Code\Datasets\Heart Attack Analysis and Prediction\heart.csv"

heart_attack = pd.read_csv(file_path)

heart_attack.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


time: 16 ms (started: 2023-12-15 10:53:50 -05:00)


In [70]:
#Defining mapping function to map multiple inputs of the dataset
def gender_map(x):
    return x.map({'M': 1, 'F': 0, 'Male' : 1, "Female" : 0, '1' : 1, '0' : 0})

def hemisphere_map(x):
    return x.map({'Southern Hemisphere' : -1, 'Northern Hemisphere' : 1})

def countries_map(x):
    return x.map({'Argentina' : -1, 'Brazil' : -1, 'China' : -1, 'Colombia' : -1,
    'India' : -1, 'Nigeria' : -1, 'South Africa' : -1, 'South Korea' : -1, 'Thailand' : -1,
    'Vietnam' : -1, 'Australia' : 1, 'Canada' : 1, 'France' : 1, 'Germany' : 1,
    'Italy' : 1, 'Japan' : 1, 'New Zealand' : 1, 'Spain' : 1, 'United Kingdom' : 1,
    'United States' : 1})

def diet_map(x):
    return x.map({'Unhealthy' : -1, 'Average' : 0, 'Healthy' : 1})

def continent_map(x):
    return x.map({'Asia' : 0, 'Africa' : 1, 'Europe' : 2, 'North America' : 3,
    'South America' : 4, 'Australia' : 5})

time: 0 ns (started: 2023-12-15 10:53:50 -05:00)


In [71]:
# heart_attack['Sex'] = heart_attack[['Sex']].apply(gender_map)
# heart_attack['Hemisphere'] = heart_attack[['Hemisphere']].apply(hemisphere_map)
# heart_attack['Country'] = heart_attack[['Country']].apply(countries_map)
# heart_attack['Diet'] = heart_attack[['Diet']].apply(diet_map)
# heart_attack['Continent'] = heart_attack[['Continent']].apply(continent_map)

# unique_sex = heart_attack['Sex'].unique()
# print(unique_sex)
# unique = heart_attack['Hemisphere'].unique()
# print(unique)
# unique_country = heart_attack['Country'].unique()
# print(unique_country)
# unique_diet = heart_attack['Diet'].unique()
# print(unique_diet)
# unique_continent = heart_attack['Continent'].unique()
# print(unique_continent)

# heart_attack.head()

time: 16 ms (started: 2023-12-15 10:53:50 -05:00)


In [72]:
# mode_columns = ['Stress Level','Medication Use', 'Previous Heart Problems', 'Diet', 'Physical Activity Days Per Week', 'Alcohol Consumption', 'Obesity', 'Smoking', 
#                 'Family History', 'Diabetes', 'Hemisphere', 'Continent', 'Country']

# mean_columns = ['Exercise Hours Per Week', 'Sleep Hours Per Day', 'BMI', 'Sedentary Hours Per Day']

# median_columns = ['Heart Rate', 'Income', 'Triglycerides']

# # Fill NaN values with the mode of the column for mode_columns
# for col in mode_columns:
#     heart_attack[col].fillna(heart_attack[col].mode()[0], inplace=True)

# # Fill NaN values with the median of the column for median_columns
# for col in median_columns:
#     heart_attack[col].fillna(heart_attack[col].median(), inplace=True)

# # Fill NaN values with the mean of the column for mean_columns
# for col in mean_columns:
#     heart_attack[col].fillna(heart_attack[col].mean(), inplace=True)


# unique_sex = heart_attack['Sex'].unique()
# print(unique_sex)
# unique = heart_attack['Hemisphere'].unique()
# print(unique)
# unique_country = heart_attack['Country'].unique()
# print(unique_country)
# unique_diet = heart_attack['Diet'].unique()
# print(unique_diet)
# unique_continent = heart_attack['Continent'].unique()
# print(unique_continent)


# Check for NaN values in columns after fillna operations
columns_with_nulls = heart_attack.columns[heart_attack.isnull().any()].tolist()
print("Columns with NaN values after fillna operations:")
print(columns_with_nulls)

heart_attack.head()

Columns with NaN values after fillna operations:
[]


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


time: 0 ns (started: 2023-12-15 10:53:50 -05:00)


In [73]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
    print("GPU is available.")
else:
    device = torch.device("cpu")   # Use CPU
    print("GPU is not available, using CPU.")

GPU is not available, using CPU.
time: 0 ns (started: 2023-12-15 10:53:50 -05:00)


In [74]:
#Splitting the Variables into X and Y
X = heart_attack.iloc[:, 0:12].values
Y = heart_attack.iloc[:, 13].values

# Split our Data set into Training Data and val Data.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)

# Convert the numpy arrays into tensors
Y_train_t = torch.tensor(Y_train, dtype=torch.float)
Y_val_t = torch.tensor(Y_val, dtype=torch.float)


print(Y_train_t.dtype)
print(Y_val_t.dtype)

torch.float32
torch.float32
time: 0 ns (started: 2023-12-15 10:53:50 -05:00)


In [75]:
# Defining PCA function
def PCA_function(X_train, X_val, K):
  pca = PCA(n_components=K)
  X_train_PCA = pca.fit_transform(X_train)
  X_val_PCA = pca.fit_transform(X_val)
  return X_train_PCA, X_val_PCA

time: 0 ns (started: 2023-12-15 10:53:50 -05:00)


In [76]:
(X_train_PCA, X_val_PCA) = PCA_function(X_train, X_val, 2)

print("The shape of X_train after PCA is", X_train_PCA.shape)
print("The shape of X_val after PCA is", X_val_PCA.shape)

# Use standard scaling from Sklearn to scale data between - and  for better accuracy.
scale = StandardScaler()
X_train_PCA = scale.fit_transform(X_train_PCA)
X_val_PCA = scale.fit_transform(X_val_PCA)

X_train_PCA_t = torch.tensor(X_train_PCA, dtype=torch.float)
X_val_PCA_t = torch.tensor(X_val_PCA, dtype=torch.float)

print("\nThe shape of the X_train tensor is", X_train_PCA_t.shape)
print("The shape of the X_val tensor is", X_val_PCA_t.shape)

The shape of X_train after PCA is (242, 2)
The shape of X_val after PCA is (61, 2)

The shape of the X_train tensor is torch.Size([242, 2])
The shape of the X_val tensor is torch.Size([61, 2])
time: 0 ns (started: 2023-12-15 10:53:50 -05:00)


In [77]:
def model_linear(t_in, w1, w2, w3, b):
  t_p = ((w3 * t_in ** 3) + (w2 * t_in ** 2) + (w1 * t_in) + b)
  return t_p

def loss_fn(t_p, t_gnd):
  squared_diffs = (t_p - t_gnd)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_in, val_t_in,
                    train_t_out, val_t_out):
  for epoch in range(1, n_epochs + 1):

    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model_linear(train_t_in, *params)
    train_loss = loss_fn(train_t_p, train_t_out)

    val_t_p = model_linear(val_t_in, *params)
    val_loss = loss_fn(val_t_p, val_t_out)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
      f" \tValidation loss {val_loss.item():.4f}")

  return params

time: 0 ns (started: 2023-12-15 10:53:50 -05:00)


In [78]:
params = torch.tensor([1.0, 1.0, 1.0, 0.0], requires_grad=True)

learning_rate_1 = 1e-3
optimizer_1 = optim.SGD([params], lr=learning_rate_1)

print('Loss every 500 epochs using learning rate of 1e-3\n')
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1,
    params = params,
    train_t_in = X_train_PCA_t,
    val_t_in = X_val_PCA_t,
    train_t_out = Y_train_t,
    val_t_out = Y_val_t)

Loss every 500 epochs using learning rate of 1e-3

Epoch 1, 	Training loss 156.1649, 	Validation loss 55.7484
Epoch 2, 	Training loss 97.7000, 	Validation loss 36.7891
Epoch 3, 	Training loss 61.3619, 	Validation loss 24.7086
Epoch 500, 	Training loss 0.5914, 	Validation loss 0.6744
Epoch 1000, 	Training loss 0.3391, 	Validation loss 0.3766
Epoch 1500, 	Training loss 0.2652, 	Validation loss 0.2809
Epoch 2000, 	Training loss 0.2420, 	Validation loss 0.2461
Epoch 2500, 	Training loss 0.2342, 	Validation loss 0.2322
Epoch 3000, 	Training loss 0.2314, 	Validation loss 0.2261
Epoch 3500, 	Training loss 0.2304, 	Validation loss 0.2233
Epoch 4000, 	Training loss 0.2301, 	Validation loss 0.2219
Epoch 4500, 	Training loss 0.2299, 	Validation loss 0.2212
Epoch 5000, 	Training loss 0.2299, 	Validation loss 0.2208


tensor([-0.1566,  0.0099,  0.0056,  0.5233], requires_grad=True)

time: 1.44 s (started: 2023-12-15 10:53:50 -05:00)


In [79]:
# Getting the predictions based on the training set using the linear model
model_output = model_linear(X_train_PCA_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_np = model_output.detach().numpy() if isinstance(model_output, torch.Tensor) else model_output

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels = np.argmax(model_output_np, axis=1)

# Calculate accuracy, precision, and recall for training set
train_accuracy = accuracy_score(Y_train, predicted_labels)
train_recall = recall_score(Y_train, predicted_labels, average= 'macro', zero_division=1)
train_precision = precision_score(Y_train, predicted_labels, average = 'macro', zero_division=1)
train_F1_score = (2*train_recall*train_precision)/(train_recall+train_precision)

# Getting the predictions based on the validation set using the linear model
model_output_val = model_linear(X_val_PCA_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_val_np = model_output_val.detach().numpy() if isinstance(model_output_val, torch.Tensor) else model_output_val

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels_val = np.argmax(model_output_val_np, axis=1)

# Calculate accuracy, precision, and recall for validation set
val_accuracy = accuracy_score(Y_val, predicted_labels_val)
val_precision = precision_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_recall = recall_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_F1_score = (2*val_recall*val_precision)/(val_recall+val_precision)

# Print the calculated metrics
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Training F1 Score: {train_F1_score:.4f}")

print(f"\nValidation Accuracy: {val_accuracy:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Validation F1 Score: {val_F1_score:.4f}")


Training Accuracy: 0.6157
Training Precision: 0.6142
Training Recall: 0.6148
Training F1 Score: 0.6145

Validation Accuracy: 0.5410
Validation Precision: 0.5460
Validation Recall: 0.5463
Validation F1 Score: 0.5461
time: 16 ms (started: 2023-12-15 10:53:52 -05:00)


In [80]:
#Sequential Model

model_seq = nn.Sequential(
    nn.Linear(2, 1024),
    nn.Tanh(),
    nn.Linear(1024, 2048),
    nn.Tanh(),
    nn.Linear(2048, 512),
    nn.Tanh(),
    nn.Linear(512, 1)
            )
model_seq

Sequential(
  (0): Linear(in_features=2, out_features=1024, bias=True)
  (1): Tanh()
  (2): Linear(in_features=1024, out_features=2048, bias=True)
  (3): Tanh()
  (4): Linear(in_features=2048, out_features=512, bias=True)
  (5): Tanh()
  (6): Linear(in_features=512, out_features=1, bias=True)
)

time: 16 ms (started: 2023-12-15 10:53:52 -05:00)


In [81]:
sum([p.numel() for p in model_seq.parameters()])

3151873

time: 0 ns (started: 2023-12-15 10:53:52 -05:00)


In [82]:
#Let's define our training loop that will be used

def training_loop_seq(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()


        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

time: 0 ns (started: 2023-12-15 10:53:52 -05:00)


In [83]:
#Let's try doing some training

optimizer_1 = optim.Adam(model_seq.parameters(), lr=1e-4)

training_loop_seq(
    n_epochs = 501,
    optimizer = optimizer_1,
    model = model_seq,
    loss_fn = nn.MSELoss(),
    x_train = X_train_PCA_t,
    x_val = X_val_PCA_t,
    y_train = Y_train_t,
    y_val = Y_val_t
)

Epoch: 1, Training loss: 0.6060, Validation loss: 0.6218
Epoch: 10, Training loss: 0.2702, Validation loss: 0.2649
Epoch: 20, Training loss: 0.2063, Validation loss: 0.1951
Epoch: 30, Training loss: 0.2085, Validation loss: 0.1931
Epoch: 40, Training loss: 0.1986, Validation loss: 0.1927
Epoch: 50, Training loss: 0.1991, Validation loss: 0.1971
Epoch: 60, Training loss: 0.1983, Validation loss: 0.1956
Epoch: 70, Training loss: 0.1978, Validation loss: 0.1937
Epoch: 80, Training loss: 0.1976, Validation loss: 0.1925
Epoch: 90, Training loss: 0.1974, Validation loss: 0.1920
Epoch: 100, Training loss: 0.1973, Validation loss: 0.1919
Epoch: 110, Training loss: 0.1972, Validation loss: 0.1920
Epoch: 120, Training loss: 0.1971, Validation loss: 0.1921
Epoch: 130, Training loss: 0.1970, Validation loss: 0.1921
Epoch: 140, Training loss: 0.1969, Validation loss: 0.1921
Epoch: 150, Training loss: 0.1967, Validation loss: 0.1921
Epoch: 160, Training loss: 0.1966, Validation loss: 0.1922
Epoch: 1

In [84]:
# Function to calculate accuracy
def calculate_accuracy(predictions, targets):
    return (predictions == targets).float().mean().item()

# Function to calculate recall
def calculate_recall(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_negatives = ((predictions == 0) & (targets == 1)).sum().item()
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Function to calculate precision
def calculate_precision(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_positives = ((predictions == 1) & (targets == 0)).sum().item()
    return true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

time: 0 ns (started: 2023-12-15 10:54:07 -05:00)


In [85]:
# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_train_PCA_t)))

# Calculate accuracy, recall, precision, and F1 Score for the training set
accuracy_train = calculate_accuracy(train_predictions, Y_train_t)
recall_train = calculate_recall(train_predictions, Y_train_t)
precision_train = calculate_precision(train_predictions, Y_train_t)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Calculate predictions for the validation set
with torch.no_grad():
    train_predictions_val = torch.round(torch.sigmoid(model_seq(X_val_PCA_t)))

# Calculate accuracy, recall, precision, and F1 Score for the validation set
accuracy_val = calculate_accuracy(train_predictions_val, Y_val_t)
recall_val = calculate_recall(train_predictions_val, Y_val_t)
precision_val = calculate_precision(train_predictions_val, Y_val_t)
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)

# Print the calculated metrics
print("Training Accuracy: {:.4f}".format(accuracy_train))
print("Training Recall: {:.4f}".format(recall_train))
print("Training Precision: {:.4f}".format(precision_train))
print("Training F1 Score: {:.4f}".format(F1_score_train))

print("\nValidation Accuracy: {:.4f}".format(accuracy_val))
print("Validation Recall: {:.4f}".format(recall_val))
print("Validation Precision: {:.4f}".format(precision_val))
print("Validation F1 Score: {:.4f}".format(F1_score_val))

Training Accuracy: 0.5620
Training Recall: 1.0000
Training Precision: 0.5527
Training F1 Score: 0.7120

Validation Accuracy: 0.5902
Validation Recall: 1.0000
Validation Precision: 0.5763
Validation F1 Score: 0.7312
time: 15 ms (started: 2023-12-15 10:54:07 -05:00)


In [86]:
model_fnn = nn.Sequential(
            nn.Linear(2, 1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(2048,1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(512, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
           )

time: 15 ms (started: 2023-12-15 10:54:07 -05:00)


In [87]:
sum([p.numel() for p in model_fnn.parameters()])

4758145

time: 0 ns (started: 2023-12-15 10:54:07 -05:00)


In [88]:
learning_rate = 1

optimizer = optim.SGD(model_fnn.parameters(), lr=learning_rate)

loss_fn = nn.BCEWithLogitsLoss()

n_epochs = 1501

#This is the Training

for epoch in range(n_epochs):
    outputs = model_fnn(X_train_PCA_t)
    loss = loss_fn(outputs, Y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # Print loss every 10 epochs
      print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.694688
Epoch: 10, Loss: 0.609694
Epoch: 20, Loss: 0.598087
Epoch: 30, Loss: 0.596970
Epoch: 40, Loss: 0.593922
Epoch: 50, Loss: 0.600851
Epoch: 60, Loss: 0.591013
Epoch: 70, Loss: 0.593637
Epoch: 80, Loss: 0.589592
Epoch: 90, Loss: 0.593503
Epoch: 100, Loss: 0.599299
Epoch: 110, Loss: 0.590558
Epoch: 120, Loss: 0.580673
Epoch: 130, Loss: 0.599433
Epoch: 140, Loss: 0.585158
Epoch: 150, Loss: 0.586089
Epoch: 160, Loss: 0.593090
Epoch: 170, Loss: 0.586894
Epoch: 180, Loss: 0.581984
Epoch: 190, Loss: 0.589195
Epoch: 200, Loss: 0.575885
Epoch: 210, Loss: 0.583310
Epoch: 220, Loss: 0.585391
Epoch: 230, Loss: 0.568722
Epoch: 240, Loss: 0.588484
Epoch: 250, Loss: 0.567937
Epoch: 260, Loss: 0.600072
Epoch: 270, Loss: 0.579487
Epoch: 280, Loss: 0.574528
Epoch: 290, Loss: 0.578828
Epoch: 300, Loss: 0.564065
Epoch: 310, Loss: 0.561638
Epoch: 320, Loss: 0.563761
Epoch: 330, Loss: 0.559997
Epoch: 340, Loss: 0.556079
Epoch: 350, Loss: 0.565035
Epoch: 360, Loss: 0.561807
Epoch: 370, 

In [89]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Training Set

correct_train = 0
total_train = 0

with torch.no_grad():
    outputs_train = model_fnn(X_train_PCA_t)
    _, predicted_train = torch.max(outputs_train, dim=1)
    total_train += Y_train_t.size(0)
    correct_train += int((predicted_train == Y_train_t.squeeze()).sum())

accuracy_train = correct_train / total_train
precision_train = precision_score(Y_train_t, predicted_train, average='macro', zero_division=0)
recall_train = recall_score(Y_train_t, predicted_train, average='macro', zero_division=0)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Print the metrics
print("Training Accuracy: %f" % accuracy_train)
print("Training Precision: %f" % precision_train)
print("Training Recall: %f" % recall_train)
print("Training F1 score: %f" % F1_score_train)

Training Accuracy: 0.458678
Training Precision: 0.229339
Training Recall: 0.500000
Training F1 score: 0.314448
time: 15 ms (started: 2023-12-15 10:55:00 -05:00)


In [90]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Validation Set

correct_val = 0
total_val = 0

with torch.no_grad():
    outputs_val = model_fnn(X_val_PCA_t)
    _, predicted_val = torch.max(outputs_val, dim=1)
    total_val += Y_val_t.size(0)
    correct_val += int((predicted_val == Y_val_t.squeeze()).sum())

accuracy_val = correct_val / total_val
precision_val = precision_score(Y_val_t, predicted_val, average='macro', zero_division=0)
recall_val = recall_score(Y_val_t, predicted_val, average='macro', zero_division=0)
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)

# Print the metrics
print("Validation Accuracy: %f" % accuracy_val)
print("Training Precision: %f" % precision_val)
print("Training Recall: %f" % recall_val)
print("Training F1 score: %f" % F1_score_val)

Validation Accuracy: 0.442623
Training Precision: 0.221311
Training Recall: 0.500000
Training F1 score: 0.306818
time: 16 ms (started: 2023-12-15 10:55:00 -05:00)
